<a href="https://colab.research.google.com/github/raja-jamwal/blog-agentic-architectures/blob/main/Part_2_Reflection%2C_Tool_Use%2C_and_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

## Reflection, Tool Use, and Planning

Welcome to Part 2 of the **Agentic Design Patterns** series.

In Part 1, we learned how to structure the flow of data (Chaining, Routing, Parallelization). Now, we are going to give our agents **brains** and **hands**. We are moving from simple pipelines to systems that can think about their work and interact with the outside world.

We will cover:
1.  **Reflection:** The ability to critique and improve one's own work.
2.  **Tool Use:** Connecting the LLM to external functions (APIs, Calculators).
3.  **Planning:** Breaking down complex goals into executable steps.

### The Stack
*   **Python**
*   **LangChain**
*   **OpenAI** (GPT-4o-mini)

### The n8n Connection
*   **Reflection** = Looping the output of an AI node back into its input (often with an `If` node to prevent infinite loops).
*   **Tool Use** = The **AI Agent Node** in n8n, which allows you to drag-and-drop tools like "Calculator" or "HTTP Request".
*   **Planning** = Using an LLM to generate a JSON list of tasks, then using the `Split In Batches` node to execute them one by one.

In [1]:
# @title 1. Install Dependencies
!pip install -qU langchain langchain-openai langchain-core

import os
from getpass import getpass

# @title 2. Setup API Key
# We need to set this again as Colab runtimes reset
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize the Model
from langchain_openai import ChatOpenAI

# We use a low temperature for reasoning tasks
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Environment Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 11.9 MB/s eta 0:00:00
Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Pattern 4: Reflection (The Self-Correction Loop)

**The Problem:** LLMs are "feed-forward." They generate text token-by-token and don't look back. This often leads to code with bugs or writing that misses the point.

**The Solution:** Implement a **Generator-Critic** workflow. One step generates the content, and a second step critiques it. Ideally, a third step revises the content based on the critique.

**The Scenario:** We want to write a Python function. Instead of trusting the first draft, we will ask the model to critique its own code and then fix it.

In [5]:
# @title Reflection Implementation
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Step 1: The Generator (Draft) ---
generator_prompt = ChatPromptTemplate.from_template(
    "Write a Python function to check if a number is prime. Return ONLY the code."
)
generator_chain = generator_prompt | llm | StrOutputParser()

# --- Step 2: The Critic (Reflection) ---
critic_prompt = ChatPromptTemplate.from_template(
    """You are a Senior Software Engineer. Critique the following code.
    Look for:
    1. Efficiency issues.
    2. Edge cases (negative numbers, 0, 1).
    3. Syntax errors.

    Code:
    {code}

    Provide a concise critique."""
)
critic_chain = critic_prompt | llm | StrOutputParser()

# --- Step 3: The Revisor (Improvement) ---
revise_prompt = ChatPromptTemplate.from_template(
    """Rewrite the following code based on the critique.
    Original Code: {code}
    Critique: {critique}

    Return ONLY the improved code."""
)
revisor_chain = revise_prompt | llm | StrOutputParser()

# --- Execution Flow ---
print("1️⃣ Generating Draft...")
draft_code = generator_chain.invoke({})
print(f"Draft:\n{draft_code}\n")

print("2️⃣ Critiquing...")
critique = critic_chain.invoke({"code": draft_code})
print(f"Feedback:\n{critique}\n")

print("3️⃣ Revising...")
final_code = revisor_chain.invoke({"code": draft_code, "critique": critique})
print(f"Final Code:\n{final_code}")

1️⃣ Generating Draft...
Draft:
```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
```

2️⃣ Critiquing...
Feedback:
The provided code for the `is_prime` function is generally well-structured, but there are a few areas for improvement regarding efficiency, edge cases, and syntax.

### 1. Efficiency Issues:
- The current implementation checks all numbers from 2 to the square root of `n`. This is efficient for most cases, but it can be further optimized:
  - After checking for divisibility by 2, you can skip all even numbers by iterating through odd numbers only (i.e., starting from 3 and incrementing by 2). This reduces the number of iterations by about half for larger numbers.

### 2. Edge Cases:
- The function correctly handles negative numbers and zero by returning `False` for `n <= 1`.
- It also correctly identifies `1` as not prime. However, it does not explicitly handl

## Pattern 5: Tool Use (Function Calling)

**The Problem:** LLMs are trapped in a text box. They can't calculate accurate math, check the weather, or query a database.

**The Solution:** **Tool Binding**. We describe a function to the LLM (inputs, outputs, description). The LLM doesn't run the code; it decides *which* tool to call and *what* arguments to pass. The system then runs the tool.

**The Scenario:** We will give the LLM a simple "Calculator" tool (since LLMs are notoriously bad at math) and ask it a math question.

In [3]:
from langchain_core.tools import tool

# --- Define the Tool ---
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers together."""
    return a * b

# --- Bind Tool to Model ---
# This tells the LLM: "You have this tool available if you need it."
llm_with_tools = llm.bind_tools([multiply])

# --- Execution ---
query = "What is 45 multiplied by 12?"
response = llm_with_tools.invoke(query)

# --- Inspecting the Result ---
# The model does NOT return text. It returns a 'tool_call' object.
print(f"User Query: {query}")
print(f"AI Decision: {response.tool_calls}")

# Note: In a full agent loop (like LangGraph or n8n), we wille explore LanGraph in another series, the system would now:
# 1. Take this tool call.
# 2. Execute the Python function `multiply(45, 12)`.
# 3. Feed the result (540) back to the LLM to generate the final answer.

User Query: What is 45 multiplied by 12?
AI Decision: [{'name': 'multiply', 'args': {'a': 45, 'b': 12}, 'id': 'call_r2zTriiFlmRfyZ7PPOrokr2w', 'type': 'tool_call'}]


## Pattern 6: Planning (The Architect)

**The Problem:** If you ask an agent to "Build a website," it will fail. The task is too big. It needs to be broken down.

**The Solution:** **Chain of Thought Planning**. Before executing, the agent generates a step-by-step plan. This plan serves as a roadmap for subsequent execution steps.

**The Scenario:** We want to write a technical blog post. Instead of writing it all at once, we will first generate a **Plan (Outline)**, and then (conceptually) we would execute each step.

In [6]:
# --- The Planner ---
planner_prompt = ChatPromptTemplate.from_template(
    """You are a Content Strategist.
    Create a numbered step-by-step plan to write a blog post about: {topic}.
    The plan should have exactly 4 steps.
    Return ONLY the list."""
)

planner_chain = planner_prompt | llm | StrOutputParser()

# --- The Executor (Simulator) ---
# In a real agent, this would be a loop. Here, we simulate the first step.
executor_prompt = ChatPromptTemplate.from_template(
    """You are a Writer. Execute Step 1 of this plan:
    Plan: {plan}

    Write the content for Step 1 only."""
)
executor_chain = executor_prompt | llm | StrOutputParser()

# --- Execution ---
topic = "The Future of AI Agents"

print("1️⃣ Creating Plan...")
plan = planner_chain.invoke({"topic": topic})
print(f"Plan Generated:\n{plan}\n")

print("2️⃣ Executing Step 1...")
step_1_content = executor_chain.invoke({"plan": plan})
print(f"Result of Step 1:\n{step_1_content}")

1️⃣ Creating Plan...
Plan Generated:
1. **Research and Gather Information**: Explore current trends, advancements, and predictions related to AI agents by reviewing academic articles, industry reports, and expert opinions.

2. **Outline the Blog Post**: Create a structured outline that includes an introduction, key sections (such as definitions, current applications, future predictions, and ethical considerations), and a conclusion.

3. **Draft the Content**: Write the blog post based on the outline, ensuring to incorporate engaging language, relevant examples, and clear explanations to make complex concepts accessible to readers.

4. **Edit and Optimize for SEO**: Review the draft for clarity, coherence, and grammatical accuracy, then optimize it for search engines by including relevant keywords, meta descriptions, and internal/external links.

2️⃣ Executing Step 1...
Result of Step 1:
### Step 1: Research and Gather Information on AI Agents

To effectively explore the current landsca

## Summary

You have now added cognitive superpowers to your agent:

1.  **Reflection:** It can fix its own mistakes.
2.  **Tool Use:** It knows how to ask for help (calculators, APIs).
3.  **Planning:** It can break big mountains into climbable hills.

In **Part 3**, we will scale this up. We will look at **Multi-Agent Collaboration** (teams of agents) and **Memory** (remembering the conversation).